In [1]:
!pip install langchain langgraph


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "<paste-your-openai-key>"
model = ChatOpenAI(model="gpt-4.1")

In [4]:
from typing import Annotated, Literal, TypedDict
import operator

class AgentInput(TypedDict):
    """Simple input state for each subagent."""
    query: str

class AgentOutput(TypedDict):
    """Output from each Subagent."""
    source: str
    result: str

class Classification(TypedDict):
    """A Single routing descision: which agent to call with that query."""
    source: Literal["github","notion","slack"]
    query: str

class RouterState(TypedDict):
    query: str
    classifications: list[Classification]
    results: Annotated[list[AgentOutput], operator.add]
    final_answer: str


In [6]:
from langchain.tools import tool

@tool
def search_code(query: str, repo: str = "main") -> str:
    """Search code in github repositories."""
    return f"Found code matching '{query}' in {repo}: authentication middleware in src/auth.py"

@tool
def search_issues(query: str) -> str:
    """Search Github issues and pull requests."""
    return f"Found 3 issues matching '{query}': #142 (API auth docs), #89 (OAuth flow), #203 (token refresh)"

@tool
def search_prs(query: str) -> str:
    """Search pull requests for implementation details."""
    return f"PR #156 added JWT authentication, PR #178 updated OAuth scopes"

@tool
def search_notion(query: str) -> str:
    """Search Notion workspace for documentation."""
    return f"Found documentation: 'API Authentication Guide' - cover OAuth2 flow, API Keys, and JWT tokens"

@tool
def get_page(page_id: str) -> str:
    """Get a specific notion page by ID."""
    return f"Page Content: step-by-step authentication setup instructions"

@tool
def search_slack(query: str) -> str:
    """Search Slack messages and threads."""
    return f"Found discussions in #engineering: 'Use Bearer tokens for API auth, see docs for refresh flow'"

@tool
def get_thread(thread_id: str) -> str:
    """Get a specific slack thread."""
    return f"Thread discusses best practices for API key rotation"


In [7]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4.1")

github_agent = create_agent(
    model,
    tools=[search_code, search_issues, search_prs],
    system_prompt=(
        "You are a Github expert. Answer questions about code."
        "API refereces, and implementation details by searching"
        "repositories, issues, and pull requests."
    ),
)

notion_agent =  create_agent(
    model,
    tools=[search_notion, get_page],
    system_prompt=(
        "You are Notion expert. Answer questions about internal"
        "processes, policies, and team documentation by searching "
        "the organization's Notion workspace."
    ),
)

slack_agent =  create_agent(
    model,
    tools=[search_slack,get_thread],
    system_prompt=(
        "You are Slack Expert. Answer questions by searching"
        "relevant threads and discussions where team members have"
        "shared knowledge and solutions."
    ),
)

In [10]:
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send

router_llm = init_chat_model("openai:gpt-4.1-mini")

class ClassificationResult(BaseModel):
    """Result of classifying a user query into agent-specific sub-questions."""
    classifications: list[Classification] = Field(
        description="List of agents to invoke with their targeted sub-questions"
    )

def classify_query(state: RouterState) -> dict:
    """Classify query and determine which agents to invoke."""
    structured_llm =  router_llm.with_structured_output(ClassificationResult)

    result = structured_llm.invoke([
        {
            "role": "system",
            "content": """Analyze this query and determine which knowledge base to consult.
    For each relevant source, generate a targeted sub-question optimized for that source.

    Available sources:
    - github: Code, API references, implementation details, issues, pull requests
    - notion: Internal documentation, processes, policies, team wikis
    -slack: Team discussions, informal knwoledge sharing, recent conversations

    Return ONLY the sources that are relevant to query. Each sources should have
    a targeted sub-question optimized for that specific knwoledge domain.

    Example for "How do I autheticate API requests?":
    - github: "What authentication code exists? Search for auth middleware, JWT handling"
    - notion: "What authentication documentation exists? Look for API auth guides"
    (slack ommitted because it is not relevant for this technical question)"""
        },
        {"role": "user", "content": state["query"]}
    ])

    return {"classifications": result.classifications}

def route_to_agents(state: RouterState) -> list[Send]:
    """Fan out to agents based on classifications."""
    return [
        Send(c["source"], {"query": c["query"]})
        for c in state["classifications"]
    ]

def query_github(state: AgentInput) -> dict:
    """Query the GitHub Agent."""
    result = github_agent.invoke({
        "messages":[{"role":"user","content": state["query"]}]
    })
    return {"results": [{"source": "github", "result": result["messages"][-1].content}]}

def query_notion(state: AgentInput) -> dict:
    """Query the Notion agent."""
    result = notion_agent.invoke({
        "messages": [{"role":"user","content": state["query"]}]
    })
    return {"results": [{"source": "notion", "result":result["messages"][-1].content}]}

def query_slack(state: AgentInput) -> dict:
    """Query the Slack agent ."""
    result = slack_agent.invoke({
        "messages": [{"role": "user", "content": state["query"]}]
    })
    return {"results": [{"source": "slack", "result":result["messages"][-1].content}]}

def synthesize_results(state: RouterState) -> dict:
    """Combine results from all agents into a coherent answer."""
    if not state["results"]:
        return {"final_answer": "No results found from any knowledge source."}

    formatted = [
        f"**From {r['source'].title()}:**\n{r['result']}"
        for r in state["results"]
    ]

    synthesis_response = router_llm.invoke([
        {
            "role": "system",
            "content": f"""Synthesize these search results to answer the original question: "{state['query']}"
- Combine information from multiple sources without redundancy
- Highlight the most relevant and actionable information
- Note any discrepancies between sources
- Keep the response concise and well-organized"""
        },
        {"role": "user","content": "\n\n".join(formatted)}
    ])

    return {"final_answer": synthesis_response.content}


        

In [11]:
workflow = (
    StateGraph(RouterState)
    .add_node("classify", classify_query)
    .add_node("github", query_github)
    .add_node("notion", query_notion)
    .add_node("slack", query_slack)
    .add_node("synthesize", synthesize_results)
    .add_edge(START, "classify")
    .add_conditional_edges("classify", route_to_agents, ["github", "notion","slack"])
    .add_edge("github","synthesize")
    .add_edge("notion","synthesize")
    .add_edge("slack","synthesize")
    .add_edge("synthesize", END)
    .compile()
)

In [12]:
result = workflow.invoke({
    "query": "How do I authenticate API requests?"
})

print("original query:", result["query"])
print("\nClassifications:")
for c in result["classifications"]:
    print(f" {c['source']}: {c['query']}")

print("\n" + "=" * 60 + "\n")
print("Final Answer:")
print(result["final_answer"])

original query: How do I authenticate API requests?

Classifications:
 github: What authentication code exists? Search for auth middleware, JWT handling
 notion: What authentication documentation exists? Look for API auth guides


Final Answer:
To authenticate API requests effectively, you can use one or a combination of the following standard methods:

1. **JWT (JSON Web Token) Authentication**  
   - As seen in the provided GitHub code (src/auth.py), JWT is used within authentication middleware to validate API requests.  
   - The client sends a JWT token typically in the `Authorization` header (`Authorization: Bearer <token>`).  
   - The server middleware verifies the token's signature and claims to authenticate the user before allowing access to protected routes.

2. **OAuth2 Flow**  
   - Supported according to the Notion "API Authentication Guide," OAuth2 is suitable for delegated authorization scenarios, where your API can issue tokens after users authorize access.  
   - This 